In [9]:
import sqlite3
import time
from ortools.linear_solver import pywraplp  

def solve_mTSP_branch_and_cut(instance_id, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Instance information
    cursor.execute("SELECT nr_cities, nr_salesmen FROM instances WHERE instance_id = ?", (instance_id,))
    instance = cursor.fetchone()
    if not instance:
        print(f"Instance {instance_id} not found.")
        conn.close()
        return

    nr_cities, nr_salesmen = instance

    # City coordinates
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ?", (instance_id,))
    cities = cursor.fetchall()

    # Create the distance matrix
    distance_matrix = [[
        ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for _, x2, y2 in cities
    ] for _, x1, y1 in cities]

    # Solve with Branch and Cut
    start_time = time.time()
    solver = pywraplp.Solver.CreateSolver('CBC')

    # Variables
    x = {}
    for i in range(nr_cities + 1):
        for j in range(nr_cities + 1):
            if i != j:
                x[i, j] = solver.BoolVar(f'x[{i},{j}]')
    u = {}
    for i in range(1, nr_cities + 1):  # Exclude the depot (city 0)
        u[i] = solver.NumVar(0, nr_cities, f'u[{i}]')

    # Constraints
    # Each city (except the depot) is visited exactly once
    for i in range(1, nr_cities + 1):
        solver.Add(solver.Sum(x[j, i] for j in range(nr_cities + 1) if j != i) == 1)
        solver.Add(solver.Sum(x[i, j] for j in range(nr_cities + 1) if j != i) == 1)

    # Each salesman starts and ends at the depot
    solver.Add(solver.Sum(x[0, j] for j in range(1, nr_cities + 1)) == nr_salesmen)
    solver.Add(solver.Sum(x[j, 0] for j in range(1, nr_cities + 1)) == nr_salesmen)

    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, nr_cities + 1):
        for j in range(1, nr_cities + 1):
            if i != j:
                solver.Add(u[i] - u[j] + (nr_cities * x[i, j]) <= nr_cities - 1)

    # Objective
    solver.Minimize(solver.Sum(distance_matrix[i][j] * x[i, j] for i in range(nr_cities + 1) for j in range(nr_cities + 1) if i != j))

    # Solve
    status = solver.Solve()
    end_time = time.time()

    if status != pywraplp.Solver.OPTIMAL:
        print(f"No optimal solution found for instance {instance_id}.")
        conn.close()
        return

    # Get the optimal cost and time taken
    optimal_cost = solver.Objective().Value()
    time_taken = end_time - start_time

    # Extract routes from the solution
    routes = []
    visited = set()  # Keep track of visited cities to avoid duplicates

    for salesman_id in range(nr_salesmen):
        route = []
        current_city = 0  # Start at the depot
        while True:
            route.append(current_city)
            visited.add(current_city)
            # Find the next city for this salesman
            try:
                next_city = next(j for j in range(nr_cities + 1) if j != current_city and j not in visited and x[current_city, j].solution_value() > 0.5)
            except StopIteration:
                route.append(0)  # Return to the depot
                break  # No valid next city found
            if next_city == 0:  # Return to the depot
                route.append(next_city)
                break
            current_city = next_city
        routes.append(route)

    # Calculate load balancing
    distances = [sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1)) for route in routes]
    load_balancing = max(distances) - min(distances)

    # Insert results into the algorithms table
    cursor.execute("""
        INSERT OR REPLACE INTO algorithms (instance_id, strategy, optimal_cost, time_taken, load_balancing)
        VALUES (?, ?, ?, ?, ?)
    """, (instance_id, "Branch and Cut", optimal_cost, time_taken, load_balancing))

    # Clear previous routes
    cursor.execute("DELETE FROM routes WHERE instance_id = ? AND strategy = ?", (instance_id, "Branch and Cut"))  
    
    # Insert routes into the routes table
    for salesman_id, route in enumerate(routes):
        cursor.execute("""
            INSERT INTO routes (instance_id, strategy, salesman_id, route)
            VALUES (?, ?, ?, ?)
        """, (instance_id, "Branch and Cut", salesman_id, str(route)))

    conn.commit()
    conn.close()

    print(f"Instance {instance_id} solved using Branch and Cut.")

def solve_mTSP_branch_and_cut_for_all_instances(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Fetch all instance IDs from the instances table
    cursor.execute("SELECT instance_id FROM instances")
    instances = cursor.fetchall()

    if not instances:
        print("No instances found in the database.")
        conn.close()
        return

    # Solve for each instance
    for (instance_id,) in instances:
        solve_mTSP_branch_and_cut(instance_id, db_file)

    conn.close()
    print("All instances have been solved.")

# Example usage
solve_mTSP_branch_and_cut_for_all_instances(db_file="../train_mTSP.sqlite3")

Instance 1 solved using Branch and Cut.
Instance 2 solved using Branch and Cut.
Instance 3 solved using Branch and Cut.
Instance 4 solved using Branch and Cut.
Instance 5 solved using Branch and Cut.
Instance 6 solved using Branch and Cut.
Instance 7 solved using Branch and Cut.
Instance 8 solved using Branch and Cut.
Instance 9 solved using Branch and Cut.
Instance 10 solved using Branch and Cut.
Instance 11 solved using Branch and Cut.
Instance 12 solved using Branch and Cut.
All instances have been solved.


In [11]:
solve_mTSP_branch_and_cut(13, "../train_mTSP.sqlite3")

Instance 13 solved using Branch and Cut.
